# XGBoost


## Omajinai

In [1]:
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 6)
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "jupyterlab"

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import xgboost

/home/eunchong/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/eunchong/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning:

0.23ubuntu1 is an invalid version and will not be supported in a future release

/home/eunchong/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning:

0.1.36ubuntu1 is an invalid version and will not be supported in a future release



In [3]:
print('pandas', pd.__version__)
print('xgboost', xgboost.__version__)

pandas 1.4.1
xgboost 1.5.2


## Data

In [4]:
!wget https://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv -O newly_confirmed_cases_daily.csv

--2022-03-02 12:53:16--  https://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv
Resolving covid19.mhlw.go.jp (covid19.mhlw.go.jp)... 18.65.200.6, 18.65.200.95, 18.65.200.99, ...
Connecting to covid19.mhlw.go.jp (covid19.mhlw.go.jp)|18.65.200.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107116 (105K) [application/octet-stream]
Saving to: ‘newly_confirmed_cases_daily.csv’

newly_confirmed_cas 100%[===================>] 104.61K  --.-KB/s    in 0.02s   

2022-03-02 12:53:16 (4.58 MB/s) - ‘newly_confirmed_cases_daily.csv’ saved [107116/107116]



In [5]:
df = pd.read_csv('newly_confirmed_cases_daily.csv', index_col=0)
df.index = pd.to_datetime(df.index)
print(df.shape)
display(df.head(1))
display(df.tail(5))

(776, 48)


,ALL,Hokkaido,Aomori,Iwate,Miyagi,Akita,Yamagata,Fukushima,Ibaraki,Tochigi,...,Ehime,Kochi,Fukuoka,Saga,Nagasaki,Kumamoto,Oita,Miyazaki,Kagoshima,Okinawa
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-16,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,ALL,Hokkaido,Aomori,Iwate,Miyagi,Akita,Yamagata,Fukushima,Ibaraki,Tochigi,...,Ehime,Kochi,Fukuoka,Saga,Nagasaki,Kumamoto,Oita,Miyazaki,Kagoshima,Okinawa
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-25,65616,1930,331,388,611,267,154,278,1188,778,...,266,189,3340,392,423,615,329,265,533,753
2022-02-26,72113,2468,476,404,793,234,208,377,1202,917,...,227,189,3394,384,368,635,360,222,497,854
2022-02-27,63630,2463,421,322,556,173,198,325,1257,667,...,209,162,2887,240,332,464,291,232,426,586
2022-02-28,51272,1605,359,278,469,119,127,174,1476,425,...,183,109,2429,298,216,393,264,140,274,422
2022-03-01,65403,1480,456,289,639,299,215,294,1009,738,...,361,220,2611,371,453,633,351,226,518,1029


In [6]:
px.line(df, )

In [7]:
df_gr = df.groupby(pd.Grouper(level='Date', freq='W')).mean()
df_gr = df_gr.drop(df_gr.index[-1])
print(df_gr.shape)
display(df_gr.head(1))
display(df_gr.tail(5))

(111, 48)


,ALL,Hokkaido,Aomori,Iwate,Miyagi,Akita,Yamagata,Fukushima,Ibaraki,Tochigi,...,Ehime,Kochi,Fukuoka,Saga,Nagasaki,Kumamoto,Oita,Miyazaki,Kagoshima,Okinawa
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-19,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,ALL,Hokkaido,Aomori,Iwate,Miyagi,Akita,Yamagata,Fukushima,Ibaraki,Tochigi,...,Ehime,Kochi,Fukuoka,Saga,Nagasaki,Kumamoto,Oita,Miyazaki,Kagoshima,Okinawa
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-30,76021.428571,2218.142857,264.285714,118.000000,452.285714,200.857143,178.142857,303.857143,990.000000,584.428571,...,282.428571,139.142857,3552.571429,363.000000,437.285714,898.428571,342.571429,379.428571,485.000000,996.285714
2022-02-06,84299.142857,2965.571429,326.000000,147.857143,614.142857,188.285714,243.285714,482.857143,1372.714286,629.142857,...,254.142857,217.000000,3351.142857,482.000000,377.142857,715.571429,403.285714,336.571429,569.000000,641.857143
2022-02-13,72924.428571,2527.142857,328.142857,163.857143,673.571429,175.857143,126.142857,393.714286,1210.000000,589.285714,...,202.428571,231.142857,2696.857143,433.857143,324.857143,615.714286,425.142857,264.857143,518.428571,558.857143
2022-02-20,79478.857143,2607.000000,411.857143,233.428571,758.857143,192.142857,193.000000,347.000000,1351.142857,806.571429,...,222.142857,221.142857,3341.000000,391.000000,352.857143,635.857143,326.857143,223.428571,492.142857,603.714286
2022-02-27,66324.000000,2262.857143,402.571429,309.285714,623.285714,201.428571,184.428571,297.142857,1170.857143,687.428571,...,214.714286,165.285714,3037.000000,341.428571,342.714286,580.000000,311.000000,216.714286,438.857143,612.571429


In [8]:
px.line(df_gr,)

### Scale

In [9]:
sc = MinMaxScaler(feature_range=(0,1))
ts_all = df_gr.ALL
ts_all_sc = sc.fit_transform(ts_all.values.reshape(-1, 1)).ravel()
ts_all_sc = pd.Series(ts_all_sc, index=ts_all.index)
ts_all_sc.head(1)

Date
2020-01-19    0.0
Freq: W-SUN, dtype: float64

### FE

In [10]:
forecast_weeks = 12
# df_fe = df_gr.ALL.to_frame()
df_fe = ts_all_sc.to_frame(name='ALL')

# add new rows for forecast
index_list = pd.date_range(ts_all_sc.index[-1]+relativedelta(weeks=1), ts_all_sc.index[-1]+relativedelta(weeks=forecast_weeks), freq='W')
df_fe = pd.concat([df_fe, pd.DataFrame(index=index_list)])

# shift
for i in range(forecast_weeks, forecast_weeks+12):
    df_fe[f'ALL_{i}_week_ago'] = df_fe.ALL.shift(i)
# df_fe[f'ALL_12_week_ago'] = df_fe.ALL.shift(12)
df_fe['year'] = df_fe.index.year
df_fe['month'] = df_fe.index.month
df_fe['weekofyear'] = df_fe.index.isocalendar().week
df_fe = df_fe.astype('float')
df_fe = df_fe.loc[df_fe.ALL_23_week_ago.dropna().index, :]
df_fe

,ALL,ALL_12_week_ago,ALL_13_week_ago,ALL_14_week_ago,ALL_15_week_ago,ALL_16_week_ago,ALL_17_week_ago,ALL_18_week_ago,ALL_19_week_ago,ALL_20_week_ago,ALL_21_week_ago,ALL_22_week_ago,ALL_23_week_ago,year,month,weekofyear
2020-06-28,0.000944,0.003232,0.001324,0.000456,0.000538,0.000397,0.000187,0.000128,0.000022,0.000002,0.000011,0.000002,0.000000,2020.0,6.0,26.0
2020-07-05,0.002154,0.005893,0.003232,0.001324,0.000456,0.000538,0.000397,0.000187,0.000128,0.000022,0.000002,0.000011,0.000002,2020.0,7.0,27.0
2020-07-12,0.003580,0.005876,0.005893,0.003232,0.001324,0.000456,0.000538,0.000397,0.000187,0.000128,0.000022,0.000002,0.000011,2020.0,7.0,28.0
2020-07-19,0.005711,0.004503,0.005876,0.005893,0.003232,0.001324,0.000456,0.000538,0.000397,0.000187,0.000128,0.000022,0.000002,2020.0,7.0,29.0
2020-07-26,0.008770,0.002785,0.004503,0.005876,0.005893,0.003232,0.001324,0.000456,0.000538,0.000397,0.000187,0.000128,0.000022,2020.0,7.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-24,NaN,0.901805,0.555176,0.215907,0.055009,0.004579,0.002658,0.001759,0.001361,0.001327,0.001066,0.001607,0.002131,2022.0,4.0,16.0
2022-05-01,NaN,1.000000,0.901805,0.555176,0.215907,0.055009,0.004579,0.002658,0.001759,0.001361,0.001327,0.001066,0.001607,2022.0,5.0,17.0
2022-05-08,NaN,0.865067,1.000000,0.901805,0.555176,0.215907,0.055009,0.004579,0.002658,0.001759,0.001361,0.001327,0.001066,2022.0,5.0,18.0
2022-05-15,NaN,0.942819,0.865067,1.000000,0.901805,0.555176,0.215907,0.055009,0.004579,0.002658,0.001759,0.001361,0.001327,2022.0,5.0,19.0


### split

In [11]:
X = df_fe.drop('ALL', axis=1)
y = df_fe.ALL
print(X.shape, y.shape)

(100, 15) (100,)


## functions

In [12]:
def trainXGBoost(X_train, y_train):    
    # val
    X_val = X_train[X_train.index == X_train.index[-1]]
    X_train = X_train.drop(X_train.index[-1])
    y_val = y_train[y_train.index == y_train.index[-1]]
    y_train = y_train.drop(y_train.index[-1])
    
    params = {'n_estimators': 1000, 'random_state': 42, 'objective': 'reg:squarederror', 'booster': 'gblinear'}
    
    model = xgboost.XGBRegressor(**params)
    model.fit(
        X_train, y_train, verbose=0,
        eval_set=[(X_val, y_val)], eval_metric='rmse', early_stopping_rounds=10,
    )

    return model

## forecast

In [13]:
%%time

train_last_date = '2022-01-02'
X_train = X[X.index <= train_last_date]
y_train = y[y.index <= train_last_date]
print(X_train.shape, y_train.shape)

model = trainXGBoost(X_train, y_train)

fi = pd.DataFrame(model.coef_, index=X_train.columns, columns=['importance']).sort_values('importance')
px.bar(fi, orientation='h')

(80, 15) (80,)


/home/eunchong/.local/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



CPU times: user 3.44 s, sys: 56.3 ms, total: 3.5 s
Wall time: 378 ms


In [14]:
X_train = X[X.index <= train_last_date]
y_train = y[y.index <= train_last_date]
X_train = X[X.index <= train_last_date]
y_train = y[y.index <= train_last_date]
y_train_index = y_train.index

y_train_pred = model.predict(X_train)
y_train_pred = sc.inverse_transform(y_train_pred.reshape(-1, 1)).ravel()
y_train_pred = pd.Series(y_train_pred, index=y_train_index, name='train_pred')
y_train = sc.inverse_transform(y_train.values.reshape(-1, 1)).ravel()
y_train = pd.Series(y_train, index=y_train_index, name='train')

rmse = np.sqrt( mean_squared_error(y_train, y_train_pred) )
print('RMSE: {:.5}'.format(rmse))

X_forecast = X[X.index > train_last_date]
y_forecast = y[y.index > train_last_date]
y_forecast_index = y_forecast.index

y_forecast = model.predict(X_forecast)
y_forecast = sc.inverse_transform(y_forecast.reshape(-1, 1)).ravel()
y_forecast = pd.Series(y_forecast, index=y_forecast_index, name='forecast')

fig = px.line(pd.concat([y_train_pred, y_train, y_forecast], axis=1))
fig.update_layout({
    'xaxis': {
        'dtick': 'M1'
    }
})
fig.show()

RMSE: 5138.2


/home/eunchong/.local/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

/home/eunchong/.local/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.

